In [1]:
###Dependecies
import sqlite3
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup
import requests
from flask import Flask, render_template, redirect, url_for
from flask_pymongo import PyMongo
from pymongo import MongoClient
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
from config import skey
import json
import pprint as pp
from collections import OrderedDict 
import pymongo

In [2]:
#establich link to local database
client = MongoClient('localhost:27017')
db = client["steam_user_db"]

In [18]:
#disclude users who appeared with no gameplay time recorded from api call
steam_users_list = db['steam users'].find({'total_games_played' : { '$gt' : 0}})

In [19]:
#convert cursor object to list
temp_list = list(steam_users_list)

In [20]:
#how many users do we have?
print(len(temp_list))

880


In [43]:
#print(temp_list[0])

{'_id': '76561198045059228', 'current_account_name': 'OMANEYE', 'original_account_name': 'Alex', 'profile_url': 'https://steamcommunity.com/profiles/76561198045059228/', 'total_games_owned': 145, 'total_games_played': 113, 'user_game_data': {'4000': {'game_number': 'game0', 'hours_played': '10.75'}, '3480': {'game_number': 'game1', 'hours_played': '4.50'}, '24740': {'game_number': 'game2', 'hours_played': '1.58'}, '22370': {'game_number': 'game3', 'hours_played': '32.27'}, '22600': {'game_number': 'game4', 'hours_played': '41.57'}, '8980': {'game_number': 'game5', 'hours_played': '79.18'}, '729040': {'game_number': 'game6', 'hours_played': '36.87'}, '620': {'game_number': 'game8', 'hours_played': '16.80'}, '105600': {'game_number': 'game9', 'hours_played': '189.52'}, '70600': {'game_number': 'game10', 'hours_played': '0.38'}, '65800': {'game_number': 'game11', 'hours_played': '78.40'}, '204030': {'game_number': 'game12', 'hours_played': '14.77'}, '22380': {'game_number': 'game13', 'hou

In [90]:
#various test statements:

#print(temp_list[0]['user_game_data'].keys())
#print(temp_list[0]['user_game_data']['4000']['hours_played'])

#current_player = db['steam users'].find({'_id' : user_id_list[0]})
#print(refined_app_ids[1] in current_player[0]['user_game_data'])

False


In [35]:
#create cumulative list of users and games
user_id_list = []
app_id_list = []
for i in range(len(temp_list)):
    user_id_list.append(temp_list[i]['_id'])
    app_id_list = app_id_list + list(temp_list[i]['user_game_data'].keys())

In [37]:
#print(len(app_id_list))
#print(len(set(app_id_list)))

119957
10491


In [57]:
#drop duplicates from game list
refined_app_ids = set(app_id_list)
refined_app_ids = list(refined_app_ids)

In [44]:
#create dataframe that will be converted to sparse.matrix
matrix_df = pd.DataFrame(index = refined_app_ids, columns=user_id_list)

In [47]:
matrix_df.head()

,76561198045059228,76561197998983239,76561198098408952,76561198045590703,76561198009324737,76561198149174522,76561198132793860,76561198029633447,76561198078745756,76561197963578673,...,76561198091258238,76561198021810199,76561197975564454,76561198197818607,76561198138109818,76561197992457738,76561197963638667,76561197985410515,76561198846437915,76561198068112781
630100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
556480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
def has_played(user_id, game_id):
    current_player = db['steam users'].find({'_id' : user_id})
    #if(current_player[0]['user_game_data'].has_key(game_id) == True):
    if(game_id in current_player[0]['user_game_data']):
        return 1
    else:
        return 0

In [94]:
print(matrix_df.info)

<bound method DataFrame.info of        76561198045059228 76561197998983239 76561198098408952  \
630100                 0                 0                 0   
1300                   0                 0                 0   
556480               NaN               NaN               NaN   
212680               NaN               NaN               NaN   
27900                NaN               NaN               NaN   
...                  ...               ...               ...   
542710               NaN               NaN               NaN   
25980                NaN               NaN               NaN   
335820               NaN               NaN               NaN   
501990               NaN               NaN               NaN   
384950               NaN               NaN               NaN   

       76561198045590703 76561198009324737 76561198149174522  \
630100                 0                 0              3.03   
1300                   0                 0                 0   
556480 

In [ ]:
# values determined by size of dataframe
# this will populate the dataframe with values for each user relative to each game
# 0 if the user has no recorded playtime
# not 0 if the user does have recorded playtime
# This is an inefficient way to populate a df

for i in range(10491):
    
    #set new value for eficiency
    current_app_id = refined_app_ids[i]
    
    for id in range(880):
        
        #set new value for eficiency
        current_player_id = user_id_list[id]
        
        if(has_played(current_player_id, current_app_id)==1):
            player = db['steam users'].find({'_id' : current_player_id})
            value_to_store = player[0]['user_game_data'][current_app_id]['hours_played']
            
            matrix_df.loc[current_app_id, current_player_id] = float(value_to_store)
        else:
            matrix_df.loc[current_app_id, current_player_id] = 0

In [93]:
matrix_df.head()

,76561198045059228,76561197998983239,76561198098408952,76561198045590703,76561198009324737,76561198149174522,76561198132793860,76561198029633447,76561198078745756,76561197963578673,...,76561198091258238,76561198021810199,76561197975564454,76561198197818607,76561198138109818,76561197992457738,76561197963638667,76561197985410515,76561198846437915,76561198068112781
630100,0,0,0,0,0,3.03,0,0,0,0,...,0,0,0,0,0,0,0.37,0,0,0
1300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
556480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#scipy.sparse.csr_matrix(matrix_df.values)

# from lightfm import LightFM
# from lightfm.evaluation import precision_at_k
# from lightfm.evaluation import auc_score

# model = LightFM(learning_rate=0.05, loss='warp')

# model.fit_partial(train, epochs=10)

In [ ]:
matrix_df